In [1]:
from langsmith import Client
import os
from datetime import datetime, timedelta

In [2]:
client = Client()

langsmith_project_name = os.getenv("LANGSMITH_PROJECT")
start_time = datetime.now() - timedelta(days=1)
project_runs = client.list_runs(
    project_name=langsmith_project_name,
    start_time=start_time,
    run_type="llm")

runs_list = list(project_runs)


In [ ]:
# View trace metadata examples
print("Exploring trace metadata...")

# Get the first few runs to examine metadata
for i, run in enumerate(runs_list[:3]):
    print(f"\n--- Run {i+1} ---")
    print(f"Run ID: {run.id}")
    print(f"Run Type: {run.run_type}")
    print(f"Start Time: {run.start_time}")
    
    # Check for metadata
    if hasattr(run, 'extra') and run.extra:
        print(f"Metadata: {run.extra}")
    
    # Check for tags
    if hasattr(run, 'tags') and run.tags:
        print(f"Tags: {run.tags}")
    
    # Check inputs/outputs for metadata
    if hasattr(run, 'inputs') and run.inputs:
        print(f"Input keys: {list(run.inputs.keys())}")
    
    print("-" * 40)


In [9]:
trace_lod = [trace.dict() for trace in runs_list if trace.session_id == trace_session_id]
len(trace_lod)

27

In [8]:
trace_session_id_list = [interaction.session_id for interaction in runs_list]
trace_session_id_list

[UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800f5-f9eb-48ff-803e-35ce75300933'),
 UUID('3c0800

In [32]:
trace_thread_id_list = [interaction.metadata['thread_id'] for interaction in runs_list]
trace_thread_id = trace_thread_id_list[0]
trace_lod = [trace.dict() for trace in runs_list if trace.metadata['thread_id'] == trace_thread_id]
trace_lod[1]["extra"]["metadata"]["langgraph_node"]

'intent_classification'

In [ ]:
# Simple metadata analysis focused on environment
import pandas as pd

# Create a DataFrame with run metadata for analysis
runs_data = []
for run in runs_list:
    run_data = {
        'run_id': run.id,
        'run_type': run.run_type,
        'start_time': run.start_time,
        'end_time': run.end_time if hasattr(run, 'end_time') else None,
        'tags': run.tags if hasattr(run, 'tags') else [],
        'metadata': run.extra if hasattr(run, 'extra') else {}
    }
    
    # Extract environment-specific metadata fields
    if run.extra:
        run_data.update({
            'environment': run.extra.get('environment'),
            'is_local_testing': run.extra.get('is_local_testing'),
            'interface': run.extra.get('interface')
        })
    
    runs_data.append(run_data)

# Convert to DataFrame for easier analysis
df_runs = pd.DataFrame(runs_data)
print(f"Total runs: {len(df_runs)}")
print(f"Columns: {df_runs.columns.tolist()}")

# Show basic environment statistics
if not df_runs.empty:
    if 'interface' in df_runs.columns:
        print(f"\nInterface distribution:")
        print(df_runs['interface'].value_counts())
    
    if 'environment' in df_runs.columns:
        print(f"\nEnvironment distribution:")
        print(df_runs['environment'].value_counts())


In [ ]:
# Simple environment analysis
if 'environment' in df_runs.columns:
    print("Environment Distribution:")
    env_counts = df_runs['environment'].value_counts()
    print(env_counts)
    
    # Show local vs production split
    local_count = len(df_runs[df_runs['is_local_testing'] == True]) if 'is_local_testing' in df_runs.columns else 0
    total_count = len(df_runs)
    prod_count = total_count - local_count
    
    print(f"\nLocal testing requests: {local_count}")
    print(f"Production requests: {prod_count}")
    print(f"Local percentage: {(local_count/total_count)*100:.1f}%" if total_count > 0 else "No data")

# Filter local vs production for analysis
if 'is_local_testing' in df_runs.columns:
    local_runs = df_runs[df_runs['is_local_testing'] == True]
    prod_runs = df_runs[df_runs['is_local_testing'] == False]
    
    print(f"\nLocal runs: {len(local_runs)}")
    print(f"Production runs: {len(prod_runs)}")
    
    # Show interface breakdown
    if 'interface' in df_runs.columns:
        print(f"\nInterface breakdown:")
        print("Local:")
        print(local_runs['interface'].value_counts() if not local_runs.empty else "No local runs")
        print("Production:")
        print(prod_runs['interface'].value_counts() if not prod_runs.empty else "No production runs")


In [ ]:
first_trace = runs_list[1]
attributes = [attr for attr in dir(first_trace) if not attr.startswith('_')]
attributes


In [ ]:
first_trace.dict()